In [ ]:
pip install langchain sentence-transformers tqdm qdrant-client pandas numpy

In [ ]:
pip install langchain_community

In [ ]:
pip install -U langchain-openai

In [ ]:
!pip install --upgrade openai

In [ ]:
# Importando bibliotecas necessárias
import os
from langchain_openai import OpenAI
from langchain.chains import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import qdrant_client
from qdrant_client.models import PointStruct, VectorParams, Distance, SearchParams
import pandas as pd
import numpy as np
import json

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# ==========================================
# 1. Carregamento de Dados
# ==========================================

In [ ]:
# Upload de arquivos no pc local
from google.colab import files
uploaded = files.upload()

In [ ]:
import json

# Carrega o arquivo JSON (substitua pelo caminho correto do arquivo JSON)
with open('/content/posts.json') as f:
    posts_data = json.load(f)

# Visualiza o conteúdo do arquivo JSON, incluindo Body, CreationDate, Score e ViewCount
for post in posts_data[:5]:  # Exibe os primeiros 5 posts
    print(f"Post ID: {post['postId']}")
    print(f"Title: {post.get('title', 'No Title')}")
    print(f"Tag: {post.get('tagName', 'No Tag')}")
    print(f"Body: {post.get('body', 'No Body')}")
    print(f"Creation Date: {post.get('creationDate', 'No Creation Date')}")
    print(f"Score: {post.get('score', 'No Score')}")
    print(f"View Count: {post.get('viewCount', 'No View Count')}")
    print('-' * 50)


Post ID: 25
Title: How to use the C socket API in C++ on z/OS
Tag: c++
Body: I'm having issues getting the C sockets API to work properly in C++ on z/OS. Although I am including sys/socket.h, I still get compile time errors telling me that AF_INET is not defined. Am I missing something obvious, or is this related to the fact that being on z/OS makes my problems much more complicated? I discovered that there is a #ifdef that I'm hitting. Apparently, z/OS isn't happy unless I define which 'type' of sockets I'm using with: #define _OE_SOCKETS  Now, I personally have no idea what this _OE_SOCKETS is actually for, so if any z/OS sockets programmers are out there (all 3 of you), perhaps you could give me a rundown of how this all works? Test App: #include &lt;sys/socket.h&gt;  int main() {     return AF_INET; }  Compile/Link Output: cxx -Wc,xplink -Wl,xplink -o inet_test inet.C  './inet.C', line 5.16: CCN5274 (S) The name lookup for 'AF_INET' did not find a declaration. CCN0797(I) Compilatio

In [ ]:
# Verificar quantas linhas foram carregadas
num_linhas = len(posts_data)
print(f"Número de linhas carregadas: {num_linhas}")

Número de linhas carregadas: 46879




---

Remover depois...

In [ ]:
# Cria um DataFrame a partir dos dados JSON
df = pd.DataFrame(posts_data)

# Substituir valores NaN por strings vazias em todas as colunas
df = df.fillna('')

# Garantir que todos os valores no DataFrame sejam strings
df = df.astype(str)



---



In [ ]:
# Preparar os textos usando o título e o corpo da pergunta, garantindo que ambos existam
texts = []

for index, row in df.iterrows():
    if row['body']:  # Verifica se o corpo não está vazio
        texts.append(f"{row['title']} {row['body']}")
    else:
        print(f"Post ID {row['postId']} não possui corpo.")

# Visualiza os primeiros 5 textos gerados
for text in texts[:5]:
    print(text)

How to use the C socket API in C++ on z/OS I'm having issues getting the C sockets API to work properly in C++ on z/OS. Although I am including sys/socket.h, I still get compile time errors telling me that AF_INET is not defined. Am I missing something obvious, or is this related to the fact that being on z/OS makes my problems much more complicated? I discovered that there is a #ifdef that I'm hitting. Apparently, z/OS isn't happy unless I define which 'type' of sockets I'm using with: #define _OE_SOCKETS  Now, I personally have no idea what this _OE_SOCKETS is actually for, so if any z/OS sockets programmers are out there (all 3 of you), perhaps you could give me a rundown of how this all works? Test App: #include &lt;sys/socket.h&gt;  int main() {     return AF_INET; }  Compile/Link Output: cxx -Wc,xplink -Wl,xplink -o inet_test inet.C  './inet.C', line 5.16: CCN5274 (S) The name lookup for 'AF_INET' did not find a declaration. CCN0797(I) Compilation failed for file ./inet.C. Object

# ==========================================
# 2. Inicialização de Ferramentas e Modelos
# ==========================================

In [ ]:
# Inicializa o LLM com a chave da API OpenAI
llm = OpenAI(temperature=0, openai_api_key="")

In [ ]:
# O template do prompt
qa_prompt = PromptTemplate.from_template("Responda a pergunta usando as informações fornecidas: {context}")

# Construir o llm_chain
llm_chain = LLMChain(llm=llm, prompt=qa_prompt)

# Criar o StuffDocumentsChain para combinar documentos
combine_docs_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context"
)

<ipython-input-14-132fadf395bb>:5: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=qa_prompt)
<ipython-input-14-132fadf395bb>:8: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_documents_chain` constructor instead. See migration guide here: https://python.langchain.com/v0.2/docs/versions/migrating_chains/stuff_docs_chain/
  combine_docs_chain = StuffDocumentsChain(


In [ ]:
# Carregar o modelo de embedding do Sentence Transformers
#model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# ==========================================
# 3. Funções de Embeddings e Pré-processamento
# ==========================================

In [ ]:
# Função para gerar embeddings em lotes
def batch_encode(texts, batch_size=256):
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i + batch_size]
        batch_embeddings = model.encode(batch, show_progress_bar=False)
        embeddings.append(batch_embeddings)
    return np.vstack(embeddings)

# ==========================================
# 4. Funções de Busca e Geração de Resposta
# ==========================================

In [ ]:
# Função para realizar a busca e recuperar os documentos mais relevantes
def retrieve(query, k=5):
    # Gerar o embedding da consulta
    query_embedding = model.encode([query])[0]

    # Buscar os k documentos mais relevantes no Qdrant
    search_result = client.search(
        collection_name=collection_name,
        query_vector=query_embedding.tolist(),
        limit=k,
        search_params={"hnsw_ef": 100}  # Parâmetro HNSW
    )

    # Extrair os textos dos resultados da busca
    docs = [Document(page_content=hit.payload["text"]) for hit in search_result]

    return docs

In [ ]:
# Função para dividir o contexto em partes menores (para o modelo LLM)
def split_context(context, max_len=500):
    sentences = context.split('. ')
    chunks = []
    chunk = ''

    for sentence in sentences:
        if len(chunk) + len(sentence) + 1 <= max_len:
            chunk += sentence + '. '
        else:
            chunks.append(chunk.strip())
            chunk = sentence + '. '

    if chunk:
        chunks.append(chunk.strip())

    # Imprimir os chunks para verificação
    for i, c in enumerate(chunks):
        print(f"Chunk {i}: {c[:100]}...")  # Exibir apenas uma parte do chunk

    return chunks

In [ ]:
# Função principal para gerar a resposta
def generate_answer(query, k=5):
    try:
        # Recuperar os documentos mais relevantes usando o Qdrant
        docs = retrieve(query, k=k)

        # Verificar se documentos foram encontrados
        if not docs:
            return "Nenhum documento relevante encontrado."

        # Concatenar os textos dos documentos
        context = " ".join([doc.page_content for doc in docs])

        # Dividir o contexto se for muito grande
        context_chunks = split_context(context)

        # Iterar sobre os chunks e usar o modelo para gerar respostas
        answers = []
        for chunk in context_chunks:
            response = llm_chain.run(context=chunk)
            answers.append(response.strip())

        # Combinar as respostas e remover duplicatas
        final_answer = ' '.join(answers).strip()

        return final_answer if final_answer else "Nenhuma resposta clara foi encontrada."

    except Exception as e:
        return f"Ocorreu um erro: {str(e)}"

# ==========================================
# 5. Integração com Qdrant
# ==========================================

In [ ]:
# Gerar embeddings em lotes
embeddings = batch_encode(texts)

100%|██████████| 184/184 [1:36:27<00:00, 31.46s/it]


In [ ]:
# Inicializar o cliente do Qdrant (alterar a conexão conforme necessário)
client = qdrant_client.QdrantClient(":memory:")

In [ ]:
# Nome da coleção no Qdrant
collection_name = "relevant_posts"


In [ ]:
# Verificar se a coleção já existe, caso contrário, recriar
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=embeddings.shape[1], distance=Distance.COSINE),
)


<ipython-input-33-3bfe527a7100>:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [ ]:
# Adicionar documentos ao Qdrant
points = [
    PointStruct(id=i, vector=embedding.tolist(), payload={"text": texts[i]})
    for i, embedding in enumerate(embeddings)
]
client.upsert(collection_name=collection_name, points=points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

# ==========================================
# 6. Testando o Sistema de Perguntas e Respostas
# ==========================================

In [ ]:
# Teste de consulta
query = "What are the main benefits and challenges of migrating from C++11 to C++14?"
response = generate_answer(query)
print(f"------------------------------------------------------------------------")
print(f"Pergunta: {query}")
print(f"Resposta: {response}")
print(f"------------------------------------------------------------------------")

Chunk 0: Switching to C++11 We are going to start a long lasting project using C++ as the programming languag...
Chunk 1: When C++11 comes out, we will still be developing the project, and would like to know if it is possi...
Chunk 2: When C++11 comes out, we will still be developing the project, and would like to know if it is possi...
Chunk 3: Are there any good / brief web pages, blog posts, or even books on how C++ has changed in the past d...
Chunk 4: What are the new developments in the C++ world?  What are the BIG things - technologies, books, fram...
Chunk 5: It's not my intention to start a flame war here, please, if you have strong opinions about not likin...
------------------------------------------------------------------------
Pergunta: What are the main benefits and challenges of migrating from C++11 to C++14?
Resposta: Qual versão do C++ será utilizada no projeto? 

C++11. Sim, é possível utilizar algumas funcionalidades do novo padrão C++11 agora para acelerar o proces

In [ ]:
# Teste de consulta
query = "What should I consider when upgrading a project from C++14 to C++20?"
response = generate_answer(query)
print(f"------------------------------------------------------------------------")
print(f"Pergunta: {query}")
print(f"Resposta: {response}")
print(f"------------------------------------------------------------------------")

Chunk 0: How do I upgrade to C++17? I am using Atom as my IDE, my current __cplusplus = 201402 which is C++14...
Chunk 1: How do I upgrade to C++17 or C++20? Everything I've searched up involves using another IDE (Microsof...
Chunk 2: Is that accurate?  Is C++21 the Next Standard? I heard a lot of buzz about C++17 and C++14 even when...
Chunk 3: But now I understand that C++17 is on track to deliver (without Concepts) in the Fall of 2017, but I...
Chunk 4: Is that accurate?  How do I check for C++20 support? What is the value of __cplusplus for C++20? Rel...
Chunk 5: I'm a self taught 'programmer' who programs in C#.  A project I would like to work on would be made ...
Chunk 6: I'm using Visual Studio 2008. It's a single-form Windows application that accesses a couple of datab...
Chunk 7: Edit: I've chosen C++ for this project for reasons I didn't go into in the post, which are not entir...
Chunk 8: So, assuming C++ is the only/best option, which should I choose?  What C++0x feature wi

In [ ]:
# Teste de consulta
query = "What specific new features introduced in C++14 help improve code readability and maintainability compared to C++11?"
response = generate_answer(query)
print(f"------------------------------------------------------------------------")
print(f"Pergunta: {query}")
print(f"Resposta: {response}")
print(f"------------------------------------------------------------------------")

Chunk 0: What are the new features in C++17? C++17 is now feature complete, so unlikely to experience large c...
Chunk 1: Hundreds of proposals were put forward for C++17.  Which of those features were added to C++ in C++1...
Chunk 2: The question  In the answers of this question it is stated that the Standard has an Appendix dedicat...
Chunk 3: According to the Standard: What changes introduced in C++14 can potentially break a program written ...
Chunk 4: The question  In the answers of this question it is stated that the Standard has an Appendix dedicat...
Chunk 5: According to the Standard: What changes introduced in C++14 can potentially break a program written ...
Chunk 6: The question  In the answers of this question it is stated that the Standard has an Appendix dedicat...
Chunk 7: According to the Standard: What changes introduced in C++14 can potentially break a program written ...
Chunk 8: Some examples:  constexpr decltype thread_local auto // New usage noexcept nullptr stat